<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/nlp/Name_Generator_Basic_RNN_Demo_Learning_First_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my first implementation on RNN and tryout to see how different it works

There are some the blogs which i followed to understand

https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html


This doc is to get a very inner level working on RNN and to understand them


We are developing a Character level RNN, what this means is our network will try to predict the next character based on the previous characters 

The output for this network is not very good, as there is not much training data and its a very simple network


In [0]:
import torch
from torch import nn

import numpy as np

In [0]:
text = ['hey how are you','good i am fine','have a nice day']

In [3]:
import unicodedata
import string

# all_letters = string.ascii_letters + " .,;'"

all_letters = "".join(list(set(''.join(text))))


n_letters = len(all_letters)

print(all_letters)

yhiwuarmfnovcegd 


We define all our letters here. In current case its just all alphabets and some special characters. 

This is sample text we wil try to model

In [4]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [5]:
def one_hot_encode(line):
    tensor = torch.zeros(len(line), n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][all_letters.find(letter)] = 1
    return tensor

one_hot_encode("hello")

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

Converting all sequences to same length, this is required only if we are doing batch processing of data because we need to have tensors of fixed length. if we always process one sequence at a time this is not needed


In [6]:
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

print(input_seq)

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fin
Target Sequence: ood i am fine
Input Sequence: have a nice da
Target Sequence: ave a nice day
['hey how are yo', 'good i am fin', 'have a nice da']


In [7]:
def normalize_length(text):
  for i in range(len(text)):
      while len(text[i])<maxlen:
          text[i] += ' '
  return text

input_seq = normalize_length(input_seq)
target_seq = normalize_length(target_seq)
print(input_seq)
print(target_seq)

['hey how are yo ', 'good i am fin  ', 'have a nice da ']
['ey how are you ', 'ood i am fine  ', 'ave a nice day ']


Our RNN purpose is to predict the next character given a set of previous characters. In current case we take a sentence and remove the last character from the input seq and remove the first character from target seq. 

**RNN will automatically process then one character at a time!!??**

In [8]:
dict_size = n_letters
seq_len = maxlen
batch_size = len(text)

def one_hot_encode_batch(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape

    # we need to have all seq of same length or else cannot create this array
    # if we don't have batch_size i.e if we process one input at a time. we 
    # don't need to have sequence of same length

    tensor = torch.zeros(batch_size, seq_len, dict_size)

    # len(sequence) and seq_len will be same in case when we are standaring the lenght!

    # features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for li in range(len(sequence[i])):
          letter = sequence[i][li]
          tensor[i][li][all_letters.find(letter)] = 1
    return tensor

print(input_seq, "input seq")
input_seq_tensor = one_hot_encode_batch(input_seq, dict_size, seq_len, batch_size)
print(input_seq_tensor.shape)

['hey how are yo ', 'good i am fin  ', 'have a nice da '] input seq
torch.Size([3, 15, 17])


One hot encoding our entire batch

In [9]:
def targetTensor(seq, seq_len):
  tensor = torch.zeros(len(seq), seq_len)
  for i in range(len(seq)):
    line = seq[i]
    for li in range(len(line)):
      tensor[i][li] = all_letters.find(line[li])
    # letter_indexes.append(n_letters - 1) # EOS
    # tensor[i][-1] = n_letters - 1
  return tensor


print(targetTensor(["hello"], seq_len))
print(target_seq)
target_seq_tensor = targetTensor(target_seq, seq_len)
print(target_seq_tensor)
print(target_seq_tensor.shape)

tensor([[ 1., 13., -1., -1., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.]])
['ey how are you ', 'ood i am fine  ', 'ave a nice day ']
tensor([[13.,  0., 16.,  1., 10.,  3., 16.,  5.,  6., 13., 16.,  0., 10.,  4.,
         16.],
        [10., 10., 15., 16.,  2., 16.,  5.,  7., 16.,  8.,  2.,  9., 13., 16.,
         16.],
        [ 5., 11., 13., 16.,  5., 16.,  9.,  2., 12., 13., 16., 15.,  5.,  0.,
         16.]])
torch.Size([3, 15])


Target tensor is not one hot encoded 

In [10]:
def convert_one_hot_to_text(one_hot):
  text = ''
  for i in range(len(one_hot)):
    for li in range(len(one_hot[i])):
      if one_hot[i][li] == 1:
        text += all_letters[li]
  
  return text

def convert_index_to_text(target_seq):
  text = ''
  for i in range(len(target_seq)):
    letter_index = int(target_seq[i].numpy().item())
    # if letter_index > 0 and letter_index != n_letters - 1:
    if letter_index > 0:
      text += all_letters[letter_index]

  return text


print(convert_one_hot_to_text(input_seq_tensor[0]))  
print(convert_index_to_text(target_seq_tensor[0]))

hey how are yo 
e how are ou 


Utility functions to see if we can decode our text back properly

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [0]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)

        # print(out.shape, 'rnn output snape')
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        # print(out.shape, 'single layer output shape')
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden


In [0]:
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 1000
lr=0.01

# Define Loss, Optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [14]:
input_seq_tensor = input_seq_tensor.to(device)
target_seq_tensor = target_seq_tensor.to(device)

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(input_seq_tensor)
    output = output.to(device)
    # print(output.shape, 'NN output shape')
    # print(target_seq_tensor.long().view(-1).shape, "target seq shape")
    loss = criterion(output, target_seq_tensor.long().view(-1))
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%100 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))
        print(output.shape, 'NN output shape')
        prob = nn.functional.softmax(output[-1], dim=0).data
        text = ''
        for x in range(len(prob)):
          if prob[x] > 0:
            text += all_letters[x]

        
        print("network output" , text)


Epoch: 100/1000............. Loss: -14.0793
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 200/1000............. Loss: -27.5910
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 300/1000............. Loss: -40.8678
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 400/1000............. Loss: -54.0519
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 500/1000............. Loss: -67.1864
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 600/1000............. Loss: -80.2900
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 700/1000............. Loss: -93.3727
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovcegd 
Epoch: 800/1000............. Loss: -106.4402
torch.Size([45, 17]) NN output shape
network output yhiwuarmfnovced 
Epoch: 900/1000............. Loss: -119.4965
torch.Size([45, 17]) NN output shape
networ

In [15]:
print(input_seq_tensor.shape, 'INPUT TENSOR SHAPE')
print(target_seq_tensor[0])
print(target_seq_tensor.shape, 'TARGET TENSOR SHAPE')
print(target_seq_tensor.long().view(-1).shape, "target seq shape")

torch.Size([3, 15, 17]) INPUT TENSOR SHAPE
tensor([13.,  0., 16.,  1., 10.,  3., 16.,  5.,  6., 13., 16.,  0., 10.,  4.,
        16.])
torch.Size([3, 15]) TARGET TENSOR SHAPE
torch.Size([45]) target seq shape


One thing in this didn't make sense to me for long time. 

 

*   The target tensor is of shape 3x15  i.e batch_size 3 and 15 dim array which has indexes for every letter. We also call view(-1) to reshare the tensor to shape 45
*   The output of the NN is of shape torch.Size([45, 57]) NN output shape in this 57 is the dict size.
* We are calling NNLoss on this. 
* The target tensor has the 3 * 15 sentenses reshaped to 45 i.e all sentenses concated 
* The share 45x57 is using NNLoss function, so interally it will call a softmax and convert it to 45x1 
* Hence the Loss works. 

https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html



In [16]:
input_seq_tensor_test = one_hot_encode_batch(["good"], dict_size, 4, 1)
print(input_seq_tensor_test.shape)
print(convert_one_hot_to_text(input_seq_tensor_test[0]))

torch.Size([1, 4, 17])
good


In [17]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    # character = np.array([[char2int[c] for c in character]])

    print(character)
    # print(len(character))
    character = one_hot_encode_batch([character], dict_size, len(character), 1)
    # print(character[0].shape)
    # print(character[0])
    # print(convert_one_hot_to_text(character[0]), "converted")  
    # return

    character = character.to(device)
    
    out, hidden = model(character)

    # print(out.shape)

    prob = nn.functional.softmax(out[-1], dim=0).data


    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    print(all_letters[char_ind])

    return all_letters[char_ind], hidden

def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, "".join(chars))
        chars.append(char)

    return ''.join(chars)

sample(model, 15, 'good')

good
o
goodo
o
goodoo
o
goodooo
o
goodoooo
o
goodooooo
o
goodoooooo
o
goodooooooo
o
goodoooooooo
o
goodooooooooo
o
goodoooooooooo
o


'goodooooooooooo'